In [115]:
# ─── Instalação de dependências ───────────────────────────────────────────────
!pip install -q google-genai gradio schedule tinydb google-api-python-client google-auth-httplib2 google-auth-oauthlib google-adk

import os
import time
import schedule
from datetime import datetime, date
from threading import Thread
import webbrowser

import gradio as gr
from google import genai
from google.genai import types
from tinydb import TinyDB, Query
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

# ─── CONFIGURAÇÕES GERAIS ──────────────────────────────────────────────────────
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

# ─── BANCO DE DADOS (TinyDB) ────────────────────────────────────────────────────
db = TinyDB('senior_hub_db.json')
history_table = db.table('history')
reminder_table = db.table('reminders')

# ─── SITES ÚTEIS (DEMONSTRATIVO) ─────────────────────────────────────────────────────
useful_websites = [
    {'name': 'INSS', 'icon': '🏠'},
    {'name': 'SUS', 'icon': '⚕️'},
    {'name': 'Receitas', 'icon': '🍳'},
    {'name': 'Relaxar', 'icon': '🧘‍♀️'},
    {'name': 'Gov.br', 'icon': '🌐'},
    {'name': 'Receita Federal', 'icon': '💰'},
    {'name': 'Embrapa', 'icon': '🌱'},
    {'name': 'Globoplay', 'icon': '📺'},
    {'name': 'Prefeitura SP', 'icon': '🏙️'},
    {'name': 'DPU', 'icon': '⚖️'}
]

# ─── AUXILIAR ABRIR URL (INUTILIZADO) ───────────────────────────────────────────
def open_url(url):
    webbrowser.open_new_tab(url)

# ─── FUNÇÕES DO CHATBOT ─────────────────────────────────────────────────────────
def load_history():
    rec = history_table.get(Query().id == 'chat')
    return rec['messages'] if rec else []

def save_history(msgs):
    history_table.upsert({'id': 'chat', 'messages': msgs}, Query().id == 'chat')

def respond_to_user(user_message, chat_history):
    history = chat_history or load_history()
    history.append((user_message, None))

    # Cria um agente para responder ao usuário com pesquisa Google E informações sobre as notícias
    response_agent = Agent(
        name='response_agent',
        model=MODEL_ID,
        instruction=f"""Você é um assistente virtual empático e atencioso, especialmente para idosos.
Sua principal função é conversar, oferecer companhia e apoio emocional.
Você pode fofocar, contar piadas e estar presente em momentos difíceis.

Você também tem acesso a uma seção de 'Notícias' onde são apresentadas as 3 notícias mais relevantes para idosos sobre um tópico específico. Se o usuário perguntar sobre notícias, mencione que você tem uma seção dedicada a isso e pode buscar as últimas informações sobre o tópico desejado. Incentive o usuário a explorar a aba 'Notícias'.

Quando necessário para responder a perguntas factuais (como horas, clima, informações gerais),
use a ferramenta google_search para obter informações relevantes e precisas.
Mantenha a conversa amigável e em português brasileiro.""",
        tools=[google_search]
    )

    agent_response = call_agent(response_agent, user_message)
    history[-1] = (user_message, agent_response)
    save_history(history)
    return history, ""

def first_interaction():
    greeting = """
Olá! Sou seu Assistente Inteligente para a Melhor Idade. 😊 Estou aqui para ser sua companhia, um ombro amigo nos momentos mais delicados, para fofocar sobre as últimas novidades, compartilhar uma piada para alegrar o seu dia e muito mais!

Nesta seção 'Utilidades', você encontrará alguns links rápidos para sites que podem ser úteis no seu dia a dia. Além disso, fique à vontade para explorar a seção 'Notícias', onde você pode encontrar as últimas informações relevantes para a melhor idade sobre diversos temas!

Sinta-se à vontade para me dizer o que está pensando ou o que gostaria de fazer. Estou aqui para você! 🤗
"""
    return [[None, greeting]]

# ─── FLUXO DE AGENTES PARA NOTÍCIAS ─────────────────────────────────────────────
def call_agent(agent: Agent, message: str) -> str:
    session_svc = InMemorySessionService()
    session_svc.create_session(app_name=agent.name, user_id='senior', session_id='session1')
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_svc)
    content = types.Content(role='user', parts=[types.Part(text=message)])
    result = ''
    for event in runner.run(user_id='senior', session_id='session1', new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text:
                    result += part.text + '\n'
    return result.strip()

# 1) Busca notícias
def news_search(topic: str) -> str:
    agent = Agent(
        name='news_search', model=MODEL_ID,
        instruction="""
Use google_search para coletar títulos e links das últimas 5 notícias do último mês sobre o tópico.
Retorne uma lista de até 5 itens no formato: Título - URL.
""",
        tools=[google_search]
    )
    return call_agent(agent, f"Tópico: {topic}\nData: {date.today()}")

# 2) Filtra e sintetiza
def news_filter(raw: str) -> str:
    agent = Agent(
        name='news_filter', model=MODEL_ID,
        instruction="""
Você recebe uma lista de notícias com título e link.
Filtre as mais relevantes (máximo 3) e sintetize cada uma em uma frase clara e concisa, focando na relevância para idosos.
Retorne tudo como texto único.
"""
    )
    return call_agent(agent, raw)

# 3) Seleciona top 3
def news_review(synth: str) -> str:
    agent = Agent(
        name='news_review', model=MODEL_ID,
        instruction="""
Revise as sínteses das notícias e escolha as 3 mais importantes e impactantes para idosos.
Retorne-as numeradas.
"""
    )
    return call_agent(agent, synth)

# Função agregada de notícias
def aggregated_news(topic: str) -> str:
    try:
        raw = news_search(topic)
        filt = news_filter(raw)
        top3 = news_review(filt)
        return top3
    except Exception as e:
        return f"❌ Erro no fluxo de notícias: {e}"

# ─── LEMBRETES E AGENDADOR ─────────────────────────────────────────────────────
def schedule_reminder(title: str, time_str: str) -> str:
    try:
        _ = datetime.strptime(time_str, '%H:%M')
    except ValueError:
        return "Formato de hora inválido. Use HH:MM."
    reminder_table.insert({'title': title, 'time': time_str})
    schedule.every().day.at(time_str).do(lambda: print(f"⏰ Lembrete: {title}"))
    return f"Lembrete '{title}' agendado para {time_str}."

# Inicia scheduler em thread separada
def run_scheduler():
    while True:
        schedule.run_pending()
        time.sleep(1)
Thread(target=run_scheduler, daemon=True).start()

# ─── INTERFACE GRADIO ──────────────────────────────────────────────────────────
css = '''
body { background: #fafafa; font-family: sans-serif; }
#header { text-align: center; margin-bottom: 20px; }
.button { background: #f0f0f0; color: #333; border: 1px solid #ccc; padding: 8px 16px; border-radius: 8px; margin-bottom: 5px; text-align: left; }
.button > span { margin-right: 10px; }
.tabitem { font-size: 1.2em; font-weight: bold; }
'''

def launch_app():
    with gr.Blocks(css=css) as demo:
        gr.Markdown("## Assistente inteligente para a melhor idade 🌟", elem_id='header')
        with gr.Tabs() as tabs:
            # Aba 1: Conversa
            with gr.TabItem("💬 Conversa", id="chat_tab", elem_classes="tabitem"):
                chatbot = gr.Chatbot(value=first_interaction(), height=300)
                user_input = gr.Textbox(placeholder="Digite aqui…")
                user_input.submit(respond_to_user, inputs=[user_input, chatbot], outputs=[chatbot, user_input])

            # Aba 2: Notícias
            with gr.TabItem("📰 Notícias", id="news_tab", elem_classes="tabitem"):
                topic_input = gr.Textbox(label="Tópico", placeholder="Ex: saúde, finanças…")
                news_btn = gr.Button("🔍 Buscar Notícias", elem_classes='button')
                news_output = gr.Textbox(label='Top 3 Notícias', lines=6)
                news_btn.click(aggregated_news, inputs=[topic_input], outputs=[news_output])

            # Aba 3: Compromissos e Medicamentos (Apenas visual)
            with gr.TabItem("🗓️ Compromissos e Medicamentos", id="schedule_tab", elem_classes="tabitem"):
                gr.Markdown("### Próximos Compromissos")
                gr.Button("Adicionar Compromisso", elem_classes='button')
                gr.Textbox(label="Lista de Compromissos", value="Nenhum compromisso agendado.", lines=3)

                gr.Markdown("### Medicamentos")
                gr.Button("Adicionar Medicamento", elem_classes='button')
                gr.Textbox(label="Lista de Medicamentos", value="Nenhum medicamento cadastrado.", lines=3)

            # Aba 4: Utilidades
            with gr.TabItem("🔗 Utilidades", id="utils_tab", elem_classes="tabitem"):
                for site in useful_websites:
                    gr.Button(f"{site['icon']} {site['name']}", elem_classes='button')

        demo.launch()

if __name__ == '__main__':
    launch_app()

<ipython-input-115-96997da37b4b>:193: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(value=first_interaction(), height=300)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f8aee9b3acc9afe2de.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
